In [ ]:
!pip install --upgrade -q gspread
!pip install krippendorff
!pip install statsmodels

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import gspread
import pandas as pd
from google.colab import auth
auth.authenticate_user()
from google.auth import default

creds, _ = default()
gc = gspread.authorize(creds)

In [ ]:
worksheet = gc.open('10k_same_sample_with_before_and_afters').worksheet("Combined")
rows = worksheet.get_all_values()
df = pd.DataFrame.from_records(rows)
df.columns=df.iloc[0]
df = df.drop([0])

In [ ]:
df.head()

,index (matches with orig 25k sentences),before,sentence,after,Ganning,Vaibhav,Final,Comments
1,17110,"Mohan Sarpanch Hearing ji's voice, he kept qui...",Only two days were spent.,Muslims from all the villages gathered and cam...,none,none,none,
2,19915,This Islamic ideology calls for the killing of...,"""The truth is that Islam is not like other bel...",It is a religious version of fascist antagonis...,none,culture,culture,
3,17972,When the defeated ruler Daud Khan asked for wa...,"About Akbar's Chittor victory, Abul Fazl wrote...","Along with them, after the victory from mornin...",oppression,oppression,oppression,
4,12626,You may remember that Mahmud Ghaznavi invaded ...,"Similarly, many Muslim invaders had come to In...","""Gajwa-e-Hind"" means conquering India and esta...",oppression,oppression,oppression,
5,16108,"Therefore, the flag on the Red Fort will swing...","In the Arab countries, Hindu women should be s...",The girls of the poor and middle of the house ...,action,oppression,action,


In [ ]:
df.loc[df["Vaibhav"] == "", "Vaibhav"] = "none"
df["Vaibhav"].unique()

array(['none', 'culture', 'oppression', 'action'], dtype=object)

## Cohen Kappa

In [ ]:
from sklearn.metrics import cohen_kappa_score

def calculate_cohen_kappa(start, end):
    df_use = df.iloc[list(range(start, end))]
    # df_use = df_use.query('Vaibhav != ""')
    a1 = df_use["Ganning"]
    a2 = df_use["Vaibhav"]
    print("Cohen Kappa for rows " + str(start) + "-" + str(end) + ": " + str(round(cohen_kappa_score(a1, a2), 2)))

def calculate_num_differences(start, end):
    df_use = df.iloc[list(range(start, end))]
    # df_use = df_use.query('Vaibhav != ""')
    num_diff = len(df_use.query('Ganning != Vaibhav'))
    
    print("Number of disagreements for rows " + str(start) + "-" + str(end) + ": " + str(num_diff))


In [ ]:
calculate_cohen_kappa(0, 100)
# calculate_num_differences(0, 100)

Cohen Kappa for rows 0-100: 0.24


In [ ]:
calculate_cohen_kappa(100, 200)
calculate_num_differences(100, 200)

Cohen Kappa for rows 100-200: 0.54
Number of disagreements for rows 100-200: 28


In [ ]:
calculate_cohen_kappa(200, 300)
calculate_num_differences(200, 300)

Cohen Kappa for rows 200-300: 0.61
Number of disagreements for rows 200-300: 24


In [ ]:
calculate_cohen_kappa(300, 400)
calculate_num_differences(300, 400)

Cohen Kappa for rows 300-400: 0.56
Number of disagreements for rows 300-400: 26


In [ ]:
calculate_cohen_kappa(400, 500)
calculate_num_differences(400, 500)

Cohen Kappa for rows 400-500: 0.71
Number of disagreements for rows 400-500: 18


In [ ]:
calculate_cohen_kappa(500, 600)
calculate_num_differences(500, 600)

Cohen Kappa for rows 500-600: 0.68
Number of disagreements for rows 500-600: 20


In [ ]:
calculate_cohen_kappa(600, 700)
calculate_num_differences(600, 700)

Cohen Kappa for rows 600-700: 0.73
Number of disagreements for rows 600-700: 17


In [ ]:
calculate_cohen_kappa(700, 800)
calculate_num_differences(700, 800)

Cohen Kappa for rows 700-800: 0.66
Number of disagreements for rows 700-800: 22


In [ ]:
calculate_cohen_kappa(800, 900)
calculate_num_differences(800, 900)

Cohen Kappa for rows 800-900: 0.83
Number of disagreements for rows 800-900: 10


In [ ]:
calculate_cohen_kappa(900, 1000)
calculate_num_differences(900, 1000)

Cohen Kappa for rows 900-1000: 0.76
Number of disagreements for rows 900-1000: 15


In [ ]:
# PHASE I
calculate_cohen_kappa(0, 400)
calculate_num_differences(0, 400)

Cohen Kappa for rows 0-400: 0.48
Number of disagreements for rows 0-400: 130


In [ ]:
# PHASE II
calculate_cohen_kappa(400, 1000)
calculate_num_differences(400, 1000)

Cohen Kappa for rows 400-1000: 0.73
Number of disagreements for rows 400-1000: 102


In [ ]:
# PHASE II
calculate_cohen_kappa(401, 700)
calculate_num_differences(400, 1000)

Cohen Kappa for rows 401-700: 0.7
Number of disagreements for rows 400-1000: 102


In [ ]:
# PHASE II
calculate_cohen_kappa(701, 1000)
calculate_num_differences(400, 1000)

Cohen Kappa for rows 701-1000: 0.75
Number of disagreements for rows 400-1000: 102


## Krippendorff Alpha

In [ ]:
import krippendorff
import numpy as np

In [ ]:
mapper = {
    "culture": 0,
    "oppression": 1,
    "action": 2,
    "none": 3,
}

In [ ]:
df_encoded = df.replace({"Ganning": mapper, "Vaibhav": mapper})

In [ ]:
df_encoded.head(2)

,index (matches with orig 25k sentences),before,sentence,after,Ganning,Vaibhav,Final,Comments
1,17110,"Mohan Sarpanch Hearing ji's voice, he kept qui...",Only two days were spent.,Muslims from all the villages gathered and cam...,3,3,none,
2,19915,This Islamic ideology calls for the killing of...,"""The truth is that Islam is not like other bel...",It is a religious version of fascist antagonis...,3,0,culture,


In [ ]:
df_encoded = df_encoded[(df_encoded["Ganning"] != "") & (df_encoded["Vaibhav"] != "")]
df_encoded

,index (matches with orig 25k sentences),before,sentence,after,Ganning,Vaibhav,Final,Comments
1,17110,"Mohan Sarpanch Hearing ji's voice, he kept qui...",Only two days were spent.,Muslims from all the villages gathered and cam...,3,3,none,
2,19915,This Islamic ideology calls for the killing of...,"""The truth is that Islam is not like other bel...",It is a religious version of fascist antagonis...,3,0,culture,
3,17972,When the defeated ruler Daud Khan asked for wa...,"About Akbar's Chittor victory, Abul Fazl wrote...","Along with them, after the victory from mornin...",1,1,oppression,
4,12626,You may remember that Mahmud Ghaznavi invaded ...,"Similarly, many Muslim invaders had come to In...","""Gajwa-e-Hind"" means conquering India and esta...",1,1,oppression,
5,16108,"Therefore, the flag on the Red Fort will swing...","In the Arab countries, Hindu women should be s...",The girls of the poor and middle of the house ...,2,1,action,
...,...,...,...,...,...,...,...,...
595,5420,Defense League!,* * If the Sri Lankan Muslims ask who is your ...,Will answer you ......_ * Islam (Muslim) ..,3,3,none,
596,12335,The car was also badly trapped in the theft ra...,Today the common housewife Ruminath is living ...,* What is their level *?,3,1,oppression,
597,14404,"On May 6, 1946, in his address to socialist ac...","In 1921 Gandhi, ignoring the opposition of nat...","Even then, in Mopala, Kerala, Muslims killed t...",3,3,none,
598,2533,"If your child is a boy, then the lady judge te...","However, flying starts with kiss or kiss on th...","Then there is the anchor, he jokes about the a...",3,3,none,


In [ ]:
value_counts = []
for i in range(500,600):
    temp_array = [0, 0, 0, 0]
    try:
        temp_array[df_encoded["Ganning"][i]] += 1
        temp_array[df_encoded["Vaibhav"][i]] += 1
    except Exception as e:
        continue
    value_counts.append(temp_array)

value_counts

[[0, 2, 0, 0],
 [2, 0, 0, 0],
 [0, 0, 2, 0],
 [0, 0, 0, 2],
 [0, 2, 0, 0],
 [0, 1, 1, 0],
 [0, 2, 0, 0],
 [2, 0, 0, 0],
 [0, 0, 1, 1],
 [0, 0, 0, 2],
 [0, 2, 0, 0],
 [0, 0, 0, 2],
 [0, 0, 0, 2],
 [0, 0, 0, 2],
 [1, 0, 0, 1],
 [0, 2, 0, 0],
 [0, 0, 0, 2],
 [0, 1, 0, 1],
 [0, 2, 0, 0],
 [1, 0, 1, 0],
 [0, 0, 2, 0],
 [0, 0, 0, 2],
 [0, 2, 0, 0],
 [0, 0, 0, 2],
 [0, 2, 0, 0],
 [0, 0, 0, 2],
 [0, 2, 0, 0],
 [0, 0, 0, 2],
 [0, 2, 0, 0],
 [0, 1, 0, 1],
 [0, 1, 0, 1],
 [1, 1, 0, 0],
 [0, 0, 0, 2],
 [0, 0, 0, 2],
 [0, 0, 0, 2],
 [0, 0, 0, 2],
 [2, 0, 0, 0],
 [2, 0, 0, 0],
 [0, 0, 0, 2],
 [1, 0, 0, 1],
 [0, 0, 0, 2],
 [0, 2, 0, 0],
 [1, 0, 0, 1],
 [0, 1, 0, 1],
 [1, 0, 0, 1],
 [0, 2, 0, 0],
 [0, 0, 2, 0],
 [0, 0, 0, 2],
 [0, 0, 0, 2],
 [0, 2, 0, 0],
 [0, 0, 0, 2],
 [0, 0, 0, 2],
 [0, 0, 1, 1],
 [0, 1, 0, 1],
 [0, 0, 0, 2],
 [0, 0, 0, 2],
 [0, 1, 0, 1],
 [0, 0, 0, 2],
 [0, 0, 2, 0],
 [0, 2, 0, 0],
 [0, 0, 0, 2],
 [0, 2, 0, 0],
 [0, 0, 0, 2],
 [0, 2, 0, 0],
 [0, 0, 0, 2],
 [0, 2, 0, 0],
 [0, 0, 0,

In [ ]:
krippendorff.alpha(value_counts=value_counts)

0.6817031442975441

## Fleiss Kappa

In [ ]:
from statsmodels.stats.inter_rater import (fleiss_kappa, cohens_kappa,
                                           to_table, aggregate_raters)
from statsmodels.tools.testing import Holder

In [ ]:
fleiss_kappa(value_counts) # works using the value counts from above

0.5702524201574233